# Data Ingestion

Having raw datasets is nice, but at this point it's still really difficult to work with! It's like we wanted a RollsRoyce dataset but ended up with an old 2007 Toyota Prius. (No offence to Prius owners).

Let's do some data processing and ingestion to **really** make this baby purr.

In this notebook we'll go from this:

* Image URLs pointing to... Something? Maybe it's a 404 page, maybe it's an audio file. Or a grayscale image. Who knows.
* Super nested List datatypes for representing image information

To all of this, nicely stored in your Data Lake:

* Clean images, all encoded as JPEG with the same mode (RGB)
* Image URLs pointing to these new clean images in your datalake (e.g. AWS S3), guaranteed to exist and not throw a 404
* A nice thumbnail in your table to help with quick visualization
* Clip embedding of the image, in case you want to get fancy with vector search
* All the extremely useful metadata about the image (height, width, num_channels etc)

Daft makes it super easy and performant to do all of this. Let's try it out.

In [1]:
import daft

df = daft.read_json("../data/mmc4_v1.1/uncompressed/docs_no_face_shard_0_v2.jsonl")
df.show()

╭───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬──────────────────────╮
│ url                   ┆ text_list             ┆ image_info            ┆ similarity_matrix     ┆ could_have_url_dupli │
│ ---                   ┆ ---                   ┆ ---                   ┆ ---                   ┆ cate                 │
│ Utf8                  ┆ List[Utf8]            ┆ List[Struct[image_nam ┆ List[List[Float64]]   ┆ ---                  │
│                       ┆                       ┆ e: Utf8, raw_url:     ┆                       ┆ Int64                │
│                       ┆                       ┆ Utf8,                 ┆                       ┆                      │
│                       ┆                       ┆ matched_text_index:   ┆                       ┆                      │
│                       ┆                       ┆ Int64, matched_sim:   ┆                       ┆                      │
│                       ┆                       ┆ Float64]]             ┆                       ┆                      │
╞═══════════════════════╪═══════════════════════╪═══════════════════════╪═══════════════════════╪══════════════════════╡
│ https://gizmodo.com/r ┆ [NASA's Hubble Space  ┆ [{image_name:         ┆ [[0.2267199158668518, ┆ 0                    │
│ ecord-br…             ┆ Telescop…             ┆ f5f8113da82f.jp…      ┆ 0.18886…              ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionplatform ┆ [Our dynamic platform ┆ [{image_name:         ┆ [[0.13888245820999146 ┆ 0                    │
│ inc.com/…             ┆ harmoni…              ┆ e39fe8050561.pn…      ┆ , 0.1538…             ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.printworx ┆ [Perfect for          ┆ [{image_name:         ┆ [[0.28204163908958435 ┆ 0                    │
│ uk.com/p…             ┆ exhibitions, dis…     ┆ ce9f0026d021.pn…      ┆ , 0.1561…             ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://xplorer.com.n ┆ [In this article, I   ┆ [{image_name:         ┆ [[0.16432377696037292 ┆ 0                    │
│ g/2019/0…             ┆ have pick…            ┆ 2b05a3b4c88e.jp…      ┆ , 0.1356…             ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://www.rejcanayna ┆ [I can't believe it's ┆ [{image_name:         ┆ [[0.16788014769554138 ┆ 0                    │
│ y.com/20…             ┆ already…              ┆ e7e97bef7dac.jp…      ┆ , 0.1680…             ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.billingsh ┆ [Commercial Space on  ┆ [{image_name:         ┆ [[0.23804685473442078 ┆ 0                    │
│ omesfors…             ┆ Red Lodg…             ┆ b10d840ab85f.jp…      ┆ , 0.2319…             ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.acornishm ┆ [I have seen so many  ┆ [{image_name:         ┆ [[0.14934484660625458 ┆ 0                    │
│ um.com/r…             ┆ people w…             ┆ 3dfe7f1b8889.jp…      ┆ , 0.1899…             ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.michaelja ┆ [​We've Moved to a New ┆ [{image_name:         ┆ [[0.20271681249141693 ┆ 0                    │
│ balee.co…             ┆ Locati…               ┆ 28233104a82c.jp…      ┆ , 0.2157…             ┆                      │
╰─────

In [2]:
df = df.explode("image_info", "similarity_matrix")
df.show()

╭───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬──────────────────────╮
│ url                   ┆ text_list             ┆ image_info            ┆ similarity_matrix     ┆ could_have_url_dupli │
│ ---                   ┆ ---                   ┆ ---                   ┆ ---                   ┆ cate                 │
│ Utf8                  ┆ List[Utf8]            ┆ Struct[image_name:    ┆ List[Float64]         ┆ ---                  │
│                       ┆                       ┆ Utf8, raw_url: Utf8,  ┆                       ┆ Int64                │
│                       ┆                       ┆ matched_text_index:   ┆                       ┆                      │
│                       ┆                       ┆ Int64, matched_sim:   ┆                       ┆                      │
│                       ┆                       ┆ Float64]              ┆                       ┆                      │
╞═══════════════════════╪═══════════════════════╪═══════════════════════╪═══════════════════════╪══════════════════════╡
│ https://gizmodo.com/r ┆ [NASA's Hubble Space  ┆ {image_name:          ┆ [0.2267199158668518,  ┆ 0                    │
│ ecord-br…             ┆ Telescop…             ┆ f5f8113da82f.jpg…     ┆ 0.188863…             ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionplatform ┆ [Our dynamic platform ┆ {image_name:          ┆ [0.13888245820999146, ┆ 0                    │
│ inc.com/…             ┆ harmoni…              ┆ e39fe8050561.png…     ┆ 0.15384…              ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionplatform ┆ [Our dynamic platform ┆ {image_name:          ┆ [0.24126467108726501, ┆ 0                    │
│ inc.com/…             ┆ harmoni…              ┆ 42532abf37cf.png…     ┆ 0.22723…              ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionplatform ┆ [Our dynamic platform ┆ {image_name:          ┆ [0.22902636229991913, ┆ 0                    │
│ inc.com/…             ┆ harmoni…              ┆ 6e3961dc4767.jpg…     ┆ 0.18699…              ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.printworx ┆ [Perfect for          ┆ {image_name:          ┆ [0.28204163908958435, ┆ 0                    │
│ uk.com/p…             ┆ exhibitions, dis…     ┆ ce9f0026d021.png…     ┆ 0.15614…              ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.printworx ┆ [Perfect for          ┆ {image_name:          ┆ [0.2898246943950653,  ┆ 0                    │
│ uk.com/p…             ┆ exhibitions, dis…     ┆ a41d8c7a197f.png…     ┆ 0.169467…             ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://xplorer.com.n ┆ [In this article, I   ┆ {image_name:          ┆ [0.16432377696037292, ┆ 0                    │
│ g/2019/0…             ┆ have pick…            ┆ 2b05a3b4c88e.jpg…     ┆ 0.13561…              ┆                      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://www.rejcanayna ┆ [I can't believe it's ┆ {image_name:          ┆ [0.16788014769554138, ┆ 0                    │
│ y.com/20…             ┆ already…              ┆ e7e97bef7dac.jpg…     ┆ 0.16807…              ┆                      │
╰───────────────────────┴───────────────────────┴───────────────────────┴───────────────────────┴──────────────────────╯

(Showi

In [3]:
df = df.with_columns({key: df[f"image_info.{key}"] for key in ["image_name", "raw_url", "matched_text_index", "matched_sim"]})
df = df.exclude("image_info")
df.show()

╭─────────────────┬─────────────────┬─────────────────┬────────────┬─────────────────┬────────────────┬────────────────╮
│ url             ┆ text_list       ┆ similarity_matr ┆      …     ┆ raw_url         ┆ matched_text_i ┆ matched_sim    │
│ ---             ┆ ---             ┆ ix              ┆            ┆ ---             ┆ ndex           ┆ ---            │
│ Utf8            ┆ List[Utf8]      ┆ ---             ┆ (2 hidden) ┆ Utf8            ┆ ---            ┆ Float64        │
│                 ┆                 ┆ List[Float64]   ┆            ┆                 ┆ Int64          ┆                │
╞═════════════════╪═════════════════╪═════════════════╪════════════╪═════════════════╪════════════════╪════════════════╡
│ https://gizmodo ┆ [NASA's Hubble  ┆ [0.226719915866 ┆ …          ┆ https://i.kinja ┆ 8              ┆ 0.314816385507 │
│ .com/record-br… ┆ Space Telescop… ┆ 8518, 0.188863… ┆            ┆ -img.com/gawke… ┆                ┆ 5836           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionpl ┆ [Our dynamic    ┆ [0.138882458209 ┆ …          ┆ http://visionpl ┆ 2              ┆ 0.195380777120 │
│ atforminc.com/… ┆ platform        ┆ 99146, 0.15384… ┆            ┆ atforminc.com/… ┆                ┆ 5902           │
│                 ┆ harmoni…        ┆                 ┆            ┆                 ┆                ┆                │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionpl ┆ [Our dynamic    ┆ [0.241264671087 ┆ …          ┆ http://visionpl ┆ 10             ┆ 0.293340384960 │
│ atforminc.com/… ┆ platform        ┆ 26501, 0.22723… ┆            ┆ atforminc.com/… ┆                ┆ 17456          │
│                 ┆ harmoni…        ┆                 ┆            ┆                 ┆                ┆                │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionpl ┆ [Our dynamic    ┆ [0.229026362299 ┆ …          ┆ http://visionpl ┆ 14             ┆ 0.244102969765 │
│ atforminc.com/… ┆ platform        ┆ 91913, 0.18699… ┆            ┆ atforminc.com/… ┆                ┆ 66315          │
│                 ┆ harmoni…        ┆                 ┆            ┆                 ┆                ┆                │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.pri ┆ [Perfect for    ┆ [0.282041639089 ┆ …          ┆ https://www.pri ┆ 0              ┆ 0.282041639089 │
│ ntworxuk.com/p… ┆ exhibitions,    ┆ 58435, 0.15614… ┆            ┆ ntworxuk.com/w… ┆                ┆ 58435          │
│                 ┆ dis…            ┆                 ┆            ┆                 ┆                ┆                │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.pri ┆ [Perfect for    ┆ [0.289824694395 ┆ …          ┆ https://www.pri ┆ 4              ┆ 0.318742632865 │
│ ntworxuk.com/p… ┆ exhibitions,    ┆ 0653, 0.169467… ┆            ┆ ntworxuk.com/w… ┆                ┆ 90576          │
│                 ┆ dis…            ┆                 ┆            ┆                 ┆                ┆                │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://xplorer ┆ [In this        ┆ [0.164323776960 ┆ …          ┆ https://xplorer ┆ 8              ┆ 0.176796704530 │
│ .com.ng/2019/0… ┆ article, I have ┆ 37292, 0.13561… ┆            ┆ net.files.word… ┆                ┆ 71594          │
│                 ┆ pick…           ┆                 ┆            ┆                 ┆                ┆                │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http:

## Downloading images

Let's try downloading some of these images to display and see what we're working with!

In [4]:
df.with_column("image", df["raw_url"].url.download().image.decode()).show()

ScanWithTask-MapPartition-LocalLimit [Stage:8]:   0%|          | 0/1 [00:00<?, ?it/s]

FileNotFoundError: File: http://visionplatforminc.com/wp-content/uploads/2018/11/monitor-for-Business-Strategy.jpg not found
HTTP status client error (404 Not Found) for url (http://visionplatforminc.com/wp-content/uploads/2018/11/monitor-for-Business-Strategy.jpg)

**Error!!!!**

Oops, what happened here?

Turns out, most of these datasets are very messy and some of these images don't even exist anymore on those servers. That's why you see that we get a 404 error when running the URL download operation.

Even more egregiously, sometimes when we download data from the URL, the data itself might not be a valid image!

We can get around this by passing `on_error="null"` into Daft, which causes Daft to log the error but proceed with execution and return a `null` entry for URLs that cannot be parsed into images.

In [5]:
df = df.with_column("image", df["raw_url"].url.download(on_error="null").image.decode(on_error="null"))

df.show()

╭──────────────────┬─────────────────┬─────────────────┬────────────┬─────────────────┬─────────────────┬──────────────╮
│ url              ┆ text_list       ┆ similarity_matr ┆      …     ┆ matched_text_in ┆ matched_sim     ┆ image        │
│ ---              ┆ ---             ┆ ix              ┆            ┆ dex             ┆ ---             ┆ ---          │
│ Utf8             ┆ List[Utf8]      ┆ ---             ┆ (3 hidden) ┆ ---             ┆ Float64         ┆ Image[MIXED] │
│                  ┆                 ┆ List[Float64]   ┆            ┆ Int64           ┆                 ┆              │
╞══════════════════╪═════════════════╪═════════════════╪════════════╪═════════════════╪═════════════════╪══════════════╡
│ https://gizmodo. ┆ [NASA's Hubble  ┆ [0.226719915866 ┆ …          ┆ 8               ┆ 0.3148163855075 ┆ <Image>      │
│ com/record-br…   ┆ Space Telescop… ┆ 8518, 0.188863… ┆            ┆                 ┆ 836             ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionpla ┆ [Our dynamic    ┆ [0.138882458209 ┆ …          ┆ 2               ┆ 0.1953807771205 ┆ None         │
│ tforminc.com/…   ┆ platform        ┆ 99146, 0.15384… ┆            ┆                 ┆ 902             ┆              │
│                  ┆ harmoni…        ┆                 ┆            ┆                 ┆                 ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionpla ┆ [Our dynamic    ┆ [0.241264671087 ┆ …          ┆ 10              ┆ 0.2933403849601 ┆ None         │
│ tforminc.com/…   ┆ platform        ┆ 26501, 0.22723… ┆            ┆                 ┆ 7456            ┆              │
│                  ┆ harmoni…        ┆                 ┆            ┆                 ┆                 ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionpla ┆ [Our dynamic    ┆ [0.229026362299 ┆ …          ┆ 14              ┆ 0.2441029697656 ┆ None         │
│ tforminc.com/…   ┆ platform        ┆ 91913, 0.18699… ┆            ┆                 ┆ 6315            ┆              │
│                  ┆ harmoni…        ┆                 ┆            ┆                 ┆                 ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.prin ┆ [Perfect for    ┆ [0.282041639089 ┆ …          ┆ 0               ┆ 0.2820416390895 ┆ None         │
│ tworxuk.com/p…   ┆ exhibitions,    ┆ 58435, 0.15614… ┆            ┆                 ┆ 8435            ┆              │
│                  ┆ dis…            ┆                 ┆            ┆                 ┆                 ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.prin ┆ [Perfect for    ┆ [0.289824694395 ┆ …          ┆ 4               ┆ 0.3187426328659 ┆ None         │
│ tworxuk.com/p…   ┆ exhibitions,    ┆ 0653, 0.169467… ┆            ┆                 ┆ 0576            ┆              │
│                  ┆ dis…            ┆                 ┆            ┆                 ┆                 ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://xplorer. ┆ [In this        ┆ [0.164323776960 ┆ …          ┆ 8               ┆ 0.1767967045307 ┆ <Image>      │
│ com.ng/2019/0…   ┆ article, I have ┆ 37292, 0.13561… ┆            ┆                 ┆ 1594            ┆              │
│                  ┆ pick…           ┆                 ┆            ┆                 ┆                 ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http:

## Data processing/metadata extraction

In [6]:
# Add a column to figure out if the image is invalid
df = df.with_column("image_invalid", df["image"].is_null())

# Extract a thumbnail of the image
df = df.with_column("thumbnail", df["image"].image.resize(64, 64))

# Extract shape of image
# df = df.with_column("image_width", df["image"].image.width())
# df = df.with_column("image_height", df["image"].image.height())
# df = df.with_column("image_channels", df["image"].image.channels())
# df = df.with_column("image_mode", df["image"].image.mode())

# Normalize the image into a hardcoded mode so that everything is standardized
# df = df.with_column("image", df["image"].image.to_mode("RGB"))

df.show()

╭───────────────────┬───────────────────┬───────────────────┬────────────┬──────────────┬───────────────┬──────────────╮
│ url               ┆ text_list         ┆ similarity_matrix ┆      …     ┆ image        ┆ image_invalid ┆ thumbnail    │
│ ---               ┆ ---               ┆ ---               ┆            ┆ ---          ┆ ---           ┆ ---          │
│ Utf8              ┆ List[Utf8]        ┆ List[Float64]     ┆ (5 hidden) ┆ Image[MIXED] ┆ Boolean       ┆ Image[MIXED] │
╞═══════════════════╪═══════════════════╪═══════════════════╪════════════╪══════════════╪═══════════════╪══════════════╡
│ https://gizmodo.c ┆ [NASA's Hubble    ┆ [0.22671991586685 ┆ …          ┆ <Image>      ┆ false         ┆ <Image>      │
│ om/record-br…     ┆ Space Telescop…   ┆ 18, 0.188863…     ┆            ┆              ┆               ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionplat ┆ [Our dynamic      ┆ [0.13888245820999 ┆ …          ┆ None         ┆ true          ┆ None         │
│ forminc.com/…     ┆ platform harmoni… ┆ 146, 0.15384…     ┆            ┆              ┆               ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionplat ┆ [Our dynamic      ┆ [0.24126467108726 ┆ …          ┆ None         ┆ true          ┆ None         │
│ forminc.com/…     ┆ platform harmoni… ┆ 501, 0.22723…     ┆            ┆              ┆               ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionplat ┆ [Our dynamic      ┆ [0.22902636229991 ┆ …          ┆ None         ┆ true          ┆ None         │
│ forminc.com/…     ┆ platform harmoni… ┆ 913, 0.18699…     ┆            ┆              ┆               ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.print ┆ [Perfect for      ┆ [0.28204163908958 ┆ …          ┆ None         ┆ true          ┆ None         │
│ worxuk.com/p…     ┆ exhibitions, dis… ┆ 435, 0.15614…     ┆            ┆              ┆               ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.print ┆ [Perfect for      ┆ [0.28982469439506 ┆ …          ┆ None         ┆ true          ┆ None         │
│ worxuk.com/p…     ┆ exhibitions, dis… ┆ 53, 0.169467…     ┆            ┆              ┆               ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://xplorer.c ┆ [In this article, ┆ [0.16432377696037 ┆ …          ┆ <Image>      ┆ false         ┆ <Image>      │
│ om.ng/2019/0…     ┆ I have pick…      ┆ 292, 0.13561…     ┆            ┆              ┆               ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://www.rejcan ┆ [I can't believe  ┆ [0.16788014769554 ┆ …          ┆ <Image>      ┆ false         ┆ <Image>      │
│ aynay.com/20…     ┆ it's already…     ┆ 138, 0.16807…     ┆            ┆              ┆               ┆              │
╰───────────────────┴───────────────────┴───────────────────┴────────────┴──────────────┴───────────────┴──────────────╯

(Showing first 8 rows)

In [7]:
# Encode the image as JPEG and get the file size and upload it to get a URL
# Then, remove the original image to avoid saving the image in your Parquet file
df = df.with_column("image_uri_s3", df["image"].image.encode("jpeg").url.upload("~/code/multimodal-data-warehouse/data/images/"))

df.show()

╭──────────────────┬──────────────────┬──────────────────┬────────────┬───────────────┬──────────────┬─────────────────╮
│ url              ┆ text_list        ┆ similarity_matri ┆      …     ┆ image_invalid ┆ thumbnail    ┆ image_uri_s3    │
│ ---              ┆ ---              ┆ x                ┆            ┆ ---           ┆ ---          ┆ ---             │
│ Utf8             ┆ List[Utf8]       ┆ ---              ┆ (6 hidden) ┆ Boolean       ┆ Image[MIXED] ┆ Utf8            │
│                  ┆                  ┆ List[Float64]    ┆            ┆               ┆              ┆                 │
╞══════════════════╪══════════════════╪══════════════════╪════════════╪═══════════════╪══════════════╪═════════════════╡
│ https://gizmodo. ┆ [NASA's Hubble   ┆ [0.2267199158668 ┆ …          ┆ false         ┆ <Image>      ┆ file:///Users/j │
│ com/record-br…   ┆ Space Telescop…  ┆ 518, 0.188863…   ┆            ┆               ┆              ┆ aychia/code/mu… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionpla ┆ [Our dynamic     ┆ [0.1388824582099 ┆ …          ┆ true          ┆ None         ┆ None            │
│ tforminc.com/…   ┆ platform         ┆ 9146, 0.15384…   ┆            ┆               ┆              ┆                 │
│                  ┆ harmoni…         ┆                  ┆            ┆               ┆              ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionpla ┆ [Our dynamic     ┆ [0.2412646710872 ┆ …          ┆ true          ┆ None         ┆ None            │
│ tforminc.com/…   ┆ platform         ┆ 6501, 0.22723…   ┆            ┆               ┆              ┆                 │
│                  ┆ harmoni…         ┆                  ┆            ┆               ┆              ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http://visionpla ┆ [Our dynamic     ┆ [0.2290263622999 ┆ …          ┆ true          ┆ None         ┆ None            │
│ tforminc.com/…   ┆ platform         ┆ 1913, 0.18699…   ┆            ┆               ┆              ┆                 │
│                  ┆ harmoni…         ┆                  ┆            ┆               ┆              ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.prin ┆ [Perfect for     ┆ [0.2820416390895 ┆ …          ┆ true          ┆ None         ┆ None            │
│ tworxuk.com/p…   ┆ exhibitions,     ┆ 8435, 0.15614…   ┆            ┆               ┆              ┆                 │
│                  ┆ dis…             ┆                  ┆            ┆               ┆              ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.prin ┆ [Perfect for     ┆ [0.2898246943950 ┆ …          ┆ true          ┆ None         ┆ None            │
│ tworxuk.com/p…   ┆ exhibitions,     ┆ 653, 0.169467…   ┆            ┆               ┆              ┆                 │
│                  ┆ dis…             ┆                  ┆            ┆               ┆              ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://xplorer. ┆ [In this         ┆ [0.1643237769603 ┆ …          ┆ false         ┆ <Image>      ┆ file:///Users/j │
│ com.ng/2019/0…   ┆ article, I have  ┆ 7292, 0.13561…   ┆            ┆               ┆              ┆ aychia/code/mu… │
│                  ┆ pick…            ┆                  ┆            ┆               ┆              ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ http:

In [11]:
# Remove columns that are large to avoid saving them in your Parquet files
df = df.exclude("image", "processed_image")

In [12]:
# For the sake of demonstration, limit to 100 rows
df = df.limit(100)

# Execute the entire dataframe, and write the results to a Parquet file!
df.write_parquet("data/tables/mmc4_v1.1.parquet")

ScanWithTask-MapPartition-LocalLimit [Stage:18]:   0%|          | 0/1 [00:00<?, ?it/s]

Error occurred during url_download at index: 3 Object at location http://visionplatforminc.com/wp-content/uploads/2018/11/monitor-for-Business-Strategy.jpg not found
Details:
HTTP status client error (404 Not Found) for url (http://visionplatforminc.com/wp-content/uploads/2018/11/monitor-for-Business-Strategy.jpg) (falling back to Null)
Error occurred during url_download at index: 1 Object at location http://visionplatforminc.com/wp-content/uploads/2018/03/VP-keep-calm-and-stop-telling-lies-500x383.png not found
Details:
HTTP status client error (404 Not Found) for url (http://visionplatforminc.com/wp-content/uploads/2018/03/VP-keep-calm-and-stop-telling-lies-500x383.png) (falling back to Null)
Error occurred during url_download at index: 2 Object at location http://visionplatforminc.com/wp-content/uploads/2018/06/comparison-chart-1200x676.png not found
Details:
HTTP status client error (404 Not Found) for url (http://visionplatforminc.com/wp-content/uploads/2018/06/comparison-chart-12

DaftCoreException: DaftError::ValueError All images in a column must have the same dtype, but got: UInt16 and UInt8

## And you're done! Your data is now "ingested".

You can now reliably run analysis on your dataset:

1. Image metadata has been extracted so you can run analyses (e.g. plot a histogram of image sizes)
2. Thumbnails have been extracted (for easy visualization)
3. Images have been normalized to the same mode (for easier downstream processing)
4. Raw URL is still available (e.g. for analysis on which domains the images were retrieved from)

Now THAT's a beautiful dataset 😍😍